In [2]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
import cv2
from PIL import Image

In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\texas\.pyenv\pyenv-win\versions\3.8.10\python.exe -m pip install --upgrade pip' command.


In [4]:
from datasets import load_dataset # Hugging face datasets 

dataset = load_dataset("kmewhort/quickdraw-bins-1pct-sample")

c:\Users\texas\.pyenv\pyenv-win\versions\3.8.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (C:/Users/texas/.cache/huggingface/datasets/kmewhort___parquet/kmewhort--quickdraw-bins-1pct-sample-eaac784574b05f1b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00,  7.15it/s]


In [5]:
print(dataset)
from struct import unpack
import io

DatasetDict({
    train: Dataset({
        features: ['label', 'packed_drawing'],
        num_rows: 403410
    })
    test: Dataset({
        features: ['label', 'packed_drawing'],
        num_rows: 100853
    })
})


In [6]:
def unpack_drawing(file_handle):
    key_id, = unpack('Q', file_handle.read(8))
    country_code, = unpack('2s', file_handle.read(2))
    recognized, = unpack('b', file_handle.read(1))
    timestamp, = unpack('I', file_handle.read(4))
    n_strokes, = unpack('H', file_handle.read(2))
    image = []
    n_bytes = 17
    for i in range(n_strokes):
        n_points, = unpack('H', file_handle.read(2))
        fmt = str(n_points) + 'B'
        x = unpack(fmt, file_handle.read(n_points))
        y = unpack(fmt, file_handle.read(n_points))
        image.append((x, y))
        n_bytes += 2 + 2*n_points
    result = {
        'key_id': key_id,
        'country_code': country_code,
        'recognized': recognized,
        'timestamp': timestamp,
        'image': image,
    }
    return result

# packed bin -> RGB PIL
def binToPIL(packed_drawing):
    padding = 8
    radius = 7
    scale = (224.0-(2*padding)) / 256
    
    unpacked = unpack_drawing(io.BytesIO(packed_drawing))
    unpacked_image = unpacked['image']
    image = np.full((224,224), 255, np.uint8)
    for stroke in unpacked['image']:
        prevX = round(stroke[0][0]*scale)
        prevY = round(stroke[1][0]*scale)
        for i in range(1, len(stroke[0])):
            x = round(stroke[0][i]*scale)
            y = round(stroke[1][i]*scale)
            cv2.line(image, (padding+prevX, padding+prevY), (padding+x, padding+y), 0, radius, -1)
            prevX = x
            prevY = y
    # turn image into 28 x 28
    image = cv2.resize(image, (28, 28))
    return image
    pilImage = Image.fromarray(image).convert("RGB")     
    return pilImage


# unpack_drawing(dataset['train'][0]['features']['packed_drawing'])
# unpack_drawing(io.BytesIO(dataset['train'][0]['packed_drawing']))

In [13]:
# x_train = np.array([str(dataset['train'][i]['packed_drawing']) for i in range(len(dataset['train']))])
x_temp = np.array([dataset['train'][i]['packed_drawing'] for i in range(len(dataset['train']))])
y_temp = np.array([dataset['train'][i]['label'] for i in range(len(dataset['train']))])

print(x_temp.shape)

# randomize the data
p = np.random.permutation(len(x_temp))
x_temp = x_temp[p]
y_temp = y_temp[p]

# only use a thousand samples for now
# x_temp = x_temp[:10000]
# y_temp = y_temp[:10000]

# testing temp

(403410,)


In [14]:
# bin to PIL
# binToPIL(str(x_train[35]))
x = []
y = []
counter = 0
for i in range(len(x_temp)):
    try:
        x.append(binToPIL(x_temp[i]))
        y.append(y_temp[i])
    except:
        counter += 1
        # print(x_temp[i])

print(counter)
# to numpy array
x = np.array(x)
y = np.array(y)

# split into train and test
split = 0.8
x_train = x[:int(split * len(x))]
y_train = y[:int(split * len(y))]
x_test = x[int(split * len(x)) :]
y_test = y[int(split * len(y)) :]

20422


In [15]:
image_size = 28

x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

In [16]:
# on the fly byte tensor to np array layer
class ByteTensorToNumpyArray(layers.Layer):
    def __init__(self, **kwargs):
        super(ByteTensorToNumpyArray, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.numpy_function(func=binToPIL, inp=[inputs], Tout=tf.uint8)
    
layer = ByteTensorToNumpyArray()


In [17]:
# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, 345)
y_test = keras.utils.to_categorical(y_test, 345)

model = keras.Sequential()
# model.add(layer)
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(345, activation='softmax'))
# Train model
# adam = tf.train.AdamOptimizer()
# model.compile(loss='categorical_crossentropy',
#               optimizer=adam,
#               metrics=['top_k_categorical_accuracy'])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['top_k_categorical_accuracy'])

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 3, 3, 64)        

In [18]:
model.fit(x = x_train, y = y_train, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

#evaluate on unseen data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Epoch 1/5
1078/1078 - 123s - loss: 3.9183 - top_k_categorical_accuracy: 0.4198 - val_loss: 3.0289 - val_top_k_categorical_accuracy: 0.6127 - 123s/epoch - 114ms/step
Epoch 2/5
1078/1078 - 113s - loss: 2.7879 - top_k_categorical_accuracy: 0.6579 - val_loss: 2.6630 - val_top_k_categorical_accuracy: 0.6787 - 113s/epoch - 104ms/step
Epoch 3/5
1078/1078 - 107s - loss: 2.4948 - top_k_categorical_accuracy: 0.7106 - val_loss: 2.4869 - val_top_k_categorical_accuracy: 0.7148 - 107s/epoch - 99ms/step
Epoch 4/5
1078/1078 - 144s - loss: 2.3324 - top_k_categorical_accuracy: 0.7381 - val_loss: 2.3637 - val_top_k_categorical_accuracy: 0.7370 - 144s/epoch - 134ms/step
Epoch 5/5
1078/1078 - 137s - loss: 2.2286 - top_k_categorical_accuracy: 0.7558 - val_loss: 2.2919 - val_top_k_categorical_accuracy: 0.7486 - 137s/epoch - 127ms/step
Test accuarcy: 74.94%
